In [1]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlsxwriter

%matplotlib qt

# import module with layers
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model_module")

from adaptive_layer import *

In [2]:
tf.random.set_seed(5)
np.random.seed(5)

In [3]:
def illustrate_train(name, y_label, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    epochs = range(1, len(static_f)+1)
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    plt.title(name, fontsize="xx-large")

    ax.set_ylabel(y_label, fontsize="large")
    ax.set_xlabel('Epochs', fontsize="large")

    ax.plot(epochs, static_f, label='ReLU Function')
    ax.plot(epochs, adaptive_f, label='Swish Function')
    ax.plot(epochs, adaptive_nn, label='Adaptive NN')
    ax.plot(epochs, adaptive_conv_nn, label='Adaptive Conv NN')

    ax.legend()
    
    plt.grid()
    plt.show()

In [5]:
def illustrate_test(name, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    
    x = ["ReLU Function", "Swish Function", "Adaptive NN", "Adaptive Conv NN"]
    y = [static_f, adaptive_f, adaptive_nn, adaptive_conv_nn]
    
    plt.title(name, fontsize="xx-large")
    ax.set_ylabel("mean absolute error", fontsize="large")
    
    ax.bar(x, y)
    
    plt.grid()
    plt.show()

In [6]:
workbook = xlsxwriter.Workbook('cr_measurements.xlsx')

In [7]:
def save_data(name, static_f_train, adaptive_f_train, adaptive_nn_train, adaptive_conv_nn_train, 
              static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test):
    
    epochs = len(static_f_train[0][:])
    worksheet = workbook.add_worksheet(name)
    
    worksheet.write(1, 1, name + " Training Loss")
    worksheet.write(1, 7, name + " Training Accuracy")
    worksheet.write(1, 13, name + " Test Accuracy")
    
    worksheet.write(2, 1, "Epochs")
    worksheet.write(2, 2, "ReLU Function")
    worksheet.write(2, 3, "Swish Function")
    worksheet.write(2, 4, "Adaptive NN")
    worksheet.write(2, 5, "Adaptive Conv NN")
    
    worksheet.write(2, 7, "Epochs")
    worksheet.write(2, 8, "ReLU Function")
    worksheet.write(2, 9, "Swish Function")
    worksheet.write(2, 10, "Adaptive NN")
    worksheet.write(2, 11, "Adaptive Conv NN")
    
    worksheet.write(2, 13, "ReLU Function")
    worksheet.write(2, 14, "Swish Function")
    worksheet.write(2, 15, "Adaptive NN")
    worksheet.write(2, 16, "Adaptive Conv NN")
    
    worksheet.write(3, 13, static_f_test)
    worksheet.write(3, 14, adaptive_f_test)
    worksheet.write(3, 15, adaptive_nn_test)
    worksheet.write(3, 16, adaptive_conv_nn_test)
    
    for i in range(0, epochs):
        worksheet.write(3+i, 1, i+1)
        worksheet.write(3+i, 2, static_f_train[0][i])
        worksheet.write(3+i, 3, adaptive_f_train[0][i])
        worksheet.write(3+i, 4, adaptive_nn_train[0][i])
        worksheet.write(3+i, 5, adaptive_conv_nn_train[0][i])
        
        worksheet.write(3+i, 7, i+1)
        worksheet.write(3+i, 8, static_f_train[1][i])
        worksheet.write(3+i, 9, adaptive_f_train[1][i])
        worksheet.write(3+i, 10, adaptive_nn_train[1][i])
        worksheet.write(3+i, 11, adaptive_conv_nn_train[1][i])
    
    workbook.close()

In [8]:
# dataset

x1 = tf.random.uniform(shape=(2500, 1), minval=-1, maxval=1)
x2 = tf.random.uniform(shape=(2500, 1), minval=-1, maxval=1)
x3 = tf.random.uniform(shape=(2500, 1), minval=-1, maxval=1)
x4 = tf.random.uniform(shape=(2500, 1), minval=-1, maxval=1)
x5 = tf.random.uniform(shape=(2500, 1), minval=-1, maxval=1)
x6 = tf.random.uniform(shape=(2500, 1), minval=-1, maxval=1)
x7 = tf.random.uniform(shape=(2500, 1), minval=-1, maxval=1)
x8 = tf.random.uniform(shape=(2500, 1), minval=-1, maxval=1)
x9 = tf.random.uniform(shape=(2500, 1), minval=-1, maxval=1)
x10 = tf.random.uniform(shape=(2500, 1), minval=-1, maxval=1)

x = np.array([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10])
x = np.reshape(x.transpose(), (x1.shape[0], 10))

x = tf.Variable(x)
y = np.array([(0.2*x[i][0]*x[i][1]*x[i][1]+x[i][2]+0.05*x[i][3]+2*x[i][4]+0.5*x[i][5]+0.1*x[i][7]
              -12*x[i][7]+x[i][8]*x[i][9]) for i in range(x.shape[0])], dtype=float)

x_train, x_test = x[:2000], x[2000:]
y_train, y_test = y[:2000], y[2000:]

In [9]:
EPOCHS = 50
name = "Classical regression"

# measurement repetitions
flag = measurements = 5

static_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_nn_hist = np.zeros(shape=[2, EPOCHS])
adaptive_conv_nn_hist = np.zeros(shape=[2, EPOCHS])

static_f_test = 0
adaptive_f_test = 0
adaptive_nn_test = 0
adaptive_conv_nn_test = 0

In [90]:
static_f_model = keras.Sequential([
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1, activation="linear")
])

In [91]:
adaptive_f_model = keras.Sequential([
    keras.layers.Dense(100, activation=tf.nn.swish),
    keras.layers.Dense(1, activation="linear")
])

In [92]:
# Adaptive Layer
adaptive_nn_model = AdaptiveModel([
    AdaptiveLayer(100, structure=[4, 2, 1], inner_hidden_activation=tf.nn.leaky_relu, 
                  inner_out_activation=tf.nn.relu, skip_w=1.7), 
    AdaptiveLayer(1, structure=[8, 4, 1], inner_hidden_activation=tf.nn.relu, skip_w=0.9) 
])

In [93]:
# Adaptive Layer based on 1D Conv
adaptive_conv_nn_model = AdaptiveModel([
    AdaptiveLayerConv(100, structure=[16], split=2, noise=0, skip_w=3.5, 
                      inner_hidden_activation=tf.nn.leaky_relu, inner_out_activation=tf.nn.leaky_relu), 
    AdaptiveLayerConv(1, structure=[32], split=2, noise=0, skip_w=1.5, 
                      inner_hidden_activation=tf.nn.leaky_relu) 
])

In [94]:
static_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="mse", metrics=['mean_absolute_error'])

In [95]:
adaptive_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="mse", metrics=['mean_absolute_error'])

In [96]:
adaptive_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="mse", metrics=['mean_absolute_error'])

In [97]:
adaptive_conv_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="mse", metrics=['mean_absolute_error'])

In [98]:
hist = static_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/50
32/32 [==============================] - 0s 502us/step - loss: 48.0974 - mean_absolute_error: 5.9848
Epoch 2/50
32/32 [==============================] - 0s 537us/step - loss: 44.4285 - mean_absolute_error: 5.7588
Epoch 3/50
32/32 [==============================] - 0s 497us/step - loss: 39.9770 - mean_absolute_error: 5.4650
Epoch 4/50
32/32 [==============================] - 0s 466us/step - loss: 34.1041 - mean_absolute_error: 5.0501
Epoch 5/50
32/32 [==============================] - 0s 473us/step - loss: 26.9489 - mean_absolute_error: 4.4819
Epoch 6/50
32/32 [==============================] - 0s 473us/step - loss: 19.2754 - mean_absolute_error: 3.7724
Epoch 7/50
32/32 [==============================] - 0s 462us/step - loss: 12.0443 - mean_absolute_error: 2.9520
Epoch 8/50
32/32 [==============================] - 0s 467us/step - loss: 6.3918 - mean_absolute_error: 2.1037
Epoch 9/50
32/32 [==============================] - 0s 473us/step - loss: 2.8320 - mean_absolute_error: 1

In [99]:
static_f_hist[0][:] += np.array(hist.history["loss"])/measurements
static_f_hist[1][:] += np.array(hist.history["mean_absolute_error"])/measurements

In [100]:
hist = adaptive_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/50
32/32 [==============================] - 0s 517us/step - loss: 47.2806 - mean_absolute_error: 5.9310
Epoch 2/50
32/32 [==============================] - 0s 513us/step - loss: 43.7915 - mean_absolute_error: 5.7154
Epoch 3/50
32/32 [==============================] - 0s 488us/step - loss: 39.7718 - mean_absolute_error: 5.4508
Epoch 4/50
32/32 [==============================] - 0s 474us/step - loss: 34.8855 - mean_absolute_error: 5.1124
Epoch 5/50
32/32 [==============================] - 0s 480us/step - loss: 29.2506 - mean_absolute_error: 4.6860
Epoch 6/50
32/32 [==============================] - 0s 487us/step - loss: 23.2237 - mean_absolute_error: 4.1758
Epoch 7/50
32/32 [==============================] - 0s 477us/step - loss: 17.1670 - mean_absolute_error: 3.5872
Epoch 8/50
32/32 [==============================] - 0s 488us/step - loss: 11.5838 - mean_absolute_error: 2.9359
Epoch 9/50
32/32 [==============================] - 0s 505us/step - loss: 6.9401 - mean_absolute_error: 

In [101]:
adaptive_f_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_f_hist[1][:] += np.array(hist.history["mean_absolute_error"])/measurements

In [102]:
hist = adaptive_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/50
32/32 [==============================] - 0s 1ms/step - loss: 46.4084 - mean_absolute_error: 5.8477
Epoch 2/50
32/32 [==============================] - 0s 1ms/step - loss: 31.2740 - mean_absolute_error: 4.5882
Epoch 3/50
32/32 [==============================] - 0s 1ms/step - loss: 18.3537 - mean_absolute_error: 2.9448
Epoch 4/50
32/32 [==============================] - 0s 1ms/step - loss: 11.0985 - mean_absolute_error: 2.4639
Epoch 5/50
32/32 [==============================] - 0s 1ms/step - loss: 5.0929 - mean_absolute_error: 1.8070
Epoch 6/50
32/32 [==============================] - 0s 1ms/step - loss: 1.7600 - mean_absolute_error: 1.0563
Epoch 7/50
32/32 [==============================] - 0s 1ms/step - loss: 0.6611 - mean_absolute_error: 0.6511
Epoch 8/50
32/32 [==============================] - 0s 1ms/step - loss: 0.3377 - mean_absolute_error: 0.4708
Epoch 9/50
32/32 [==============================] - 0s 1ms/step - loss: 0.2175 - mean_absolute_error: 0.3748
Epoch 10/50
32/

In [103]:
adaptive_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_nn_hist[1][:] += np.array(hist.history["mean_absolute_error"])/measurements

In [104]:
hist = adaptive_conv_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/50
32/32 [==============================] - 0s 1ms/step - loss: 44.4916 - mean_absolute_error: 5.7429
Epoch 2/50
32/32 [==============================] - 0s 1ms/step - loss: 17.8558 - mean_absolute_error: 3.3663
Epoch 3/50
32/32 [==============================] - 0s 1ms/step - loss: 0.9787 - mean_absolute_error: 0.8142
Epoch 4/50
32/32 [==============================] - 0s 1ms/step - loss: 0.4447 - mean_absolute_error: 0.5578
Epoch 5/50
32/32 [==============================] - 0s 1ms/step - loss: 0.3312 - mean_absolute_error: 0.4797
Epoch 6/50
32/32 [==============================] - 0s 1ms/step - loss: 0.2505 - mean_absolute_error: 0.4165
Epoch 7/50
32/32 [==============================] - 0s 1ms/step - loss: 0.1951 - mean_absolute_error: 0.3634
Epoch 8/50
32/32 [==============================] - 0s 1ms/step - loss: 0.1534 - mean_absolute_error: 0.3184
Epoch 9/50
32/32 [==============================] - 0s 1ms/step - loss: 0.1252 - mean_absolute_error: 0.2819
Epoch 10/50
32/32

In [105]:
adaptive_conv_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_conv_nn_hist[1][:] += np.array(hist.history["mean_absolute_error"])/measurements

In [106]:
static_f_test += static_f_model.evaluate(x_test, y_test)[1]/measurements

16/16 [==============================] - 0s 502us/step - loss: 0.0435 - mean_absolute_error: 0.1623


In [107]:
adaptive_f_test += adaptive_f_model.evaluate(x_test, y_test)[1]/measurements

16/16 [==============================] - 0s 523us/step - loss: 0.0134 - mean_absolute_error: 0.0880


In [108]:
adaptive_nn_test += adaptive_nn_model.evaluate(x_test, y_test)[1]/measurements

16/16 [==============================] - 0s 742us/step - loss: 0.0108 - mean_absolute_error: 0.0764


In [109]:
adaptive_conv_nn_test += adaptive_conv_nn_model.evaluate(x_test, y_test)[1]/measurements
flag -= 1
if(flag == 0):
    print("---------FINISH---------")

16/16 [==============================] - 0s 714us/step - loss: 0.0076 - mean_absolute_error: 0.0685
---------FINISH---------


In [110]:
print("Static Function model\nTest mean_absolute_error : ", static_f_test)
print("\nAdaptive Function model\nTest mean_absolute_error : ", adaptive_f_test)
print("\nAdaptive NN model\nTest mean_absolute_error : ", adaptive_nn_test)
print("\nAdaptive Conv NN model\nTest mean_absolute_error : ", adaptive_conv_nn_test)

Static Function model
Test mean_absolute_error :  0.16776592433452606

Adaptive Function model
Test mean_absolute_error :  0.09616599977016449

Adaptive NN model
Test mean_absolute_error :  0.07488027960062027

Adaptive Conv NN model
Test mean_absolute_error :  0.0675840362906456


In [111]:
illustrate_train(name+" Training Loss", "Loss", static_f_hist[0][:], 
                 adaptive_f_hist[0][:], adaptive_nn_hist[0][:], adaptive_conv_nn_hist[0][:])

In [112]:
illustrate_train(name + " Training Mean Absolute Error", "mean absolute error", static_f_hist[1][:], 
                 adaptive_f_hist[1][:], adaptive_nn_hist[1][:], adaptive_conv_nn_hist[1][:])

In [113]:
illustrate_test(name + " Test", 
                static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)

In [114]:
save_data(name, static_f_hist, adaptive_f_hist, adaptive_nn_hist, 
          adaptive_conv_nn_hist, static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)